In [307]:
from sklearn.model_selection import train_test_split
import pandas
import numpy
import math

In [338]:
data = pandas.read_csv("data.csv") ## Loading our dataset 
mobilesTrain, mobilesTest = train_test_split(data, test_size=0.2) ## Split it into 2 parts(train/test)
mobilesTrain.columns ## Columns of our dataset 

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [339]:
### Prices and Their Counts
mobilesTrain['price_range'].value_counts()

3    410
0    403
1    398
2    389
Name: price_range, dtype: int64

In [340]:
### Prior Probabilities
Probabilities = []
mobilesTrainRows = mobilesTrain.shape[0]
for x in range(4):
  Probabilities.append(mobilesTrain['price_range'].value_counts().get(x)/mobilesTrainRows)
Probabilities = numpy.array(Probabilities).reshape(4,1,1)
Probabilities

array([[[0.251875]],

       [[0.24875 ]],

       [[0.243125]],

       [[0.25625 ]]])

In [341]:
price_range = [0,1,2,3]
trainValues = []
for x in range(len(price_range)):
  trainValues.append((mobilesTrain.loc[mobilesTrain['price_range'] == price_range[x]]).iloc[:,0:20].to_numpy())

In [342]:
### Calculating Means
means = []
for x in range(len(price_range)):
  mobile = trainValues[x]
  mean = mobile.mean(axis=0)
  means.append(mean)
means = numpy.array(means)
means = means.reshape(4,1,20)

In [343]:
### Calculating Variances
variances = []
for x in range(len(price_range)):
  mobile = trainValues[x]
  variance = mobile.var(axis = 0)
  variances.append(variance)
variances = numpy.array(variances) 
variances = variances.reshape(4,1,20) 

In [344]:
mobilesTest.shape[0]

400

In [345]:
### Pre-Processing Test Dataset
testValues = mobilesTest.iloc[:,0:20].to_numpy().reshape(400,20)
yTest = mobilesTest.iloc[:,20]

In [346]:
### Implementation Of Gaussian Naive Bayes Classifier
predictions = (1/(numpy.sqrt(2*math.pi*variances)))*numpy.exp((1/(2*variances))*(-numpy.square(testValues - means)))
predictions = Probabilities*numpy.prod(predictions,axis=2).reshape(4,400,1)
predictions = numpy.argmax(predictions,axis=0)

In [347]:
### Accuracy ###
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
yTest = label_encoder.fit_transform(yTest)
yTest = yTest.reshape(400,1)
numpy.sum(predictions == yTest) / 400

0.82